In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import tarfile


tar = tarfile.open("drive/My Drive/data.tgz", "r:gz")
tar.extractall(path="./dataset")
tar.close()

In [0]:

import numpy as np 
from matplotlib import pyplot as p

import tensorflow as tf 

from keras.layers import Dense, Reshape, Conv2D, Conv2DTranspose, LeakyReLU , BatchNormalization, Input, Activation, Flatten
from keras.models import Model
from keras.utils import plot_model
from keras.optimizers import RMSprop


from PIL import Image 

import os

Using TensorFlow backend.


In [0]:
import imageio

class ImageHelper(object):
    def save_image(self, generated, epoch, directory):
        fig, axs = p.subplots(5, 5)
        count = 0
        for i in range(5):
            for j in range(5):
                axs[i,j].imshow(generated[count, :,:,:])
                axs[i,j].axis('off')
                count += 1
        fig.savefig("{}/{}.png".format(directory, epoch))
        p.close()
        
    def makegif(self, directory):
        filenames = np.sort(os.listdir(directory))
        filenames = [ fnm for fnm in filenames if ".png" in fnm]
    
        with imageio.get_writer(directory + '/image.gif', mode='I') as writer:
            for filename in filenames:
                image = imageio.imread(directory + filename)
                writer.append_data(image)

In [0]:

x_train = []

pth = "dataset/cropped/"

l = len(os.listdir(pth))
k = 0

for i in os.listdir(pth) : 
  
   try : 
    k = k + 1 
    print("{}%".format(k*100//l))

    x_train.append(np.asarray(Image.open(pth+i).resize((32,32),Image.ANTIALIAS))/255.0)
   
   except : 
      print("err")
      pass

In [0]:
x_train = np.asarray(x_train)

In [0]:
class dcgan : 
  
  def __init__(self,img_shape,noise_shape,x_train) : 
    
    self.img_shape = img_shape
    self.img_w = img_shape[0]
    self.img_h = img_shape[1]
    self.channels = img_shape[2]
    
    self.latent_size = 100  
    
    self.x_train = np.reshape(x_train,(-1,self.img_w,self.img_h,self.channels))
    
    self.batch_size = 64
    
    #build discriminator
    
    self.discriminator = self.build_discriminator()
    
    optimizer = RMSprop(lr=2e-4,decay=6e-8)
    self.discriminator.compile(optimizer=optimizer,loss="binary_crossentropy",metrics=["accuracy"])
    
    plot_model(self.discriminator,show_shapes=True)
    
    #build generator
    
    self.generator = self.build_generator()
    
    plot_model(self.generator,show_shapes=True)
    
    #build dcgan
    lr = 2e-4
    decay = 6e-8
    
    optimizer = RMSprop(lr=lr*0.5,decay=decay*0.5)
    
    self.discriminator.trainable = False
    input = Input(shape=(self.latent_size,),name="generator_input")
    self.model = Model(input,self.discriminator(self.generator(input)),name="dcgan")
    self.model.compile(optimizer=optimizer,loss="binary_crossentropy",metrics=["accuracy"])
    
    plot_model(self.model)
    
  def build_generator(self) : 
    
    input = Input(shape=(self.latent_size,),name="generator_input")
    
    x = Dense(np.prod(self.img_shape))(input)
    x = Reshape(self.img_shape)(x)
    x = Conv2DTranspose(128,kernel_size=(5,5),strides=1,padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(64,kernel_size=(5,5),strides=1,padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(32,kernel_size=(5,5),strides=1,padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(3,kernel_size=(5,5),strides=1,padding="same")(x)
    x = Activation("sigmoid")(x)
    
    generator = Model(input,x,name="generator")
    
    return generator
  
  def build_discriminator(self) : 
    
    input = Input(shape=self.img_shape,name="discriminator_input")
    
    x = Conv2D(32,kernel_size=(5,5),strides=2,padding="same")(input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(64,kernel_size=(5,5),strides=2,padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128,kernel_size=(5,5),strides=2,padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(256,kernel_size=(5,5),strides=1,padding="same")(input)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Flatten()(x)
    x = Dense(1)(x)
    x = Activation("sigmoid")(x)
    
    discriminator = Model(input,x,name="discriminator")
    
    return discriminator
  
  def train(self) : 
    
    train_noise = np.random.uniform(-1.0,1.0,size=[16,self.latent_size])
    
    train_size = self.x_train.shape[0]
    
    for i in range(10000) : 
      
      #train discriminator
      rand_indexs = np.random.randint(0,train_size,size=self.batch_size)
      noise = np.random.uniform(-1.0,1.0,size=[self.batch_size,self.latent_size])
      
      real_images = self.x_train[rand_indexs]
      fake_images = self.generator.predict(noise)
      #real+fake images = 1 batch of train
      x = np.concatenate((real_images,fake_images))
      y = np.ones([2*self.batch_size,1])
      y[self.batch_size:,:] = 0.0
      loss, acc = self.discriminator.train_on_batch(x,y)
      log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)
      
      #train the model
      
      noise = np.random.uniform(-1.0,1.0,size=[self.batch_size,self.latent_size])
      y = np.ones([self.batch_size,1])
      
      
      loss, acc = self.model.train_on_batch(noise,y)

      log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
      print(log)
      
      if (i + 1) % 100 == 0:
        if (i + 1) == 40000:
          show = True
        else:
          show = False
        
        self.sample_images(i)

     
     
    self.generator.save("./drive/My Drive/dcgan_generator_anime.h5")

  def sample_images(self, epoch):
      r, c = 5, 5
      noise = np.random.uniform(-1.0,1.0, (r * c, self.latent_size))
      gen_imgs = self.generator.predict(noise)

      # Rescale images 0 - 1
      gen_imgs = 0.5 * gen_imgs + 0.5

      fig, axs = p.subplots(r, c)
      cnt = 0
      for i in range(r):
          for j in range(c):
              axs[i,j].imshow(gen_imgs[cnt, :,:,:])
              axs[i,j].axis('off')
              cnt += 1
      fig.savefig("./im/%d.png" % epoch)
      p.close()

In [0]:
#dcgan = dcgan((32,32,3),100,x_train)

dcgan.train(dcgan)

In [0]:
dcgan.generator.save("./drive/My Drive/dcgan_generator_anime.h5")